download dataset  
https://s3.amazonaws.com/tripdata/index.html  
https://s3.amazonaws.com/tripdata/201809-citibike-tripdata.csv.zip  

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


In [2]:
import pandas as pd
df = pd.read_csv("/gdrive/My Drive/data/201809-citibike-tripdata.csv")

In [3]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


# Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [4]:
df.shape

(1877884, 15)

# Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [5]:
#*
df['tripduration'] = df['tripduration']/60
round(df.describe()['tripduration']['mean'], 2)

16.13

## Сколько поездок начались и закончились в той же самой станции?

In [6]:
df[df['start station id'] == df['end station id']].shape

(41364, 15)

In [7]:
#*
df[(df['start station id'] == df['end station id'])]['bikeid'].unique().shape

(9939,)

# Какой самый используемый байк(bikeid) в городе по количеству поездок?

In [8]:
df.groupby(['bikeid'])['bikeid'].agg(['count']).sort_values(by=['count'], ascending=False).head()


,count
bikeid,
33875,559
33772,549
34357,549
33868,533
33878,532


In [20]:
#*
df['bikeid'].value_counts()[:1]

33875    559
Name: bikeid, dtype: int64

# Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [9]:
df.groupby(['tripduration', 'bikeid'])[['tripduration']].mean()

tripduration
tripduration bikeid              
1.016667     14532       1.016667
             14691       1.016667
             14769       1.016667
             14798       1.016667
             14829       1.016667
...                           ...
31644.950000 33876   31644.950000
35374.350000 27911   35374.350000
39041.466667 19083   39041.466667
39369.900000 17431   39369.900000
40527.516667 17548   40527.516667

[1753156 rows x 1 columns]

In [21]:
#*
df.groupby('bikeid')['tripduration'].mean().sort_values(ascending=False)[:1]

bikeid
17548    8133.706667
Name: tripduration, dtype: float64

# Сколько строк, в которых отсутствуют данные о start station id?

In [10]:
df[df['start station id'].isnull()]['bikeid'].agg(['count'])

count    716
Name: bikeid, dtype: int64

In [22]:
#*
df[df['start station id'].isnull()].shape

(716, 17)

# Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?

In [11]:
df.groupby(['usertype'])['tripduration'].mean() / 60

usertype
Customer      0.556923
Subscriber    0.222206
Name: tripduration, dtype: float64

In [23]:
#*
round(df.groupby('usertype')['tripduration'].mean(),2)

usertype
Customer      33.42
Subscriber    13.33
Name: tripduration, dtype: float64

# Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).

# Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)

In [12]:
from geopy.distance import vincenty

In [13]:
round(df[df['start station id'] != df['end station id']].apply(lambda df: vincenty(
              (df['start station latitude'],df['start station longitude']),
              (df['end station latitude'],df['end station longitude'])
              ).km,
              axis=1).mean(),2)

1.85

In [24]:
#*
df['distance_km'] = df.apply(lambda x: vincenty((x['start station latitude'], x['start station longitude']),
                                                            (x['end station latitude'], x['end station longitude'])).kilometers, axis=1)

print(df['distance_km'].mean())

df['end station name'].value_counts()[:5]

1.8088453985540238


Pershing Square North    13834
E 17 St & Broadway       11964
West St & Chambers St    11173
Broadway & E 22 St       10962
W 21 St & 6 Ave          10431
Name: end station name, dtype: int64

# Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно

In [14]:
from datetime import datetime, time

time_from = time(18, 00)
time_till = time(20, 00)

df['starttime_time'] = df.apply(lambda x: 
          datetime.strptime(x['starttime'],'%Y-%m-%d %H:%M:%S.%f').time()
          ,axis=1)

df_start_station = df[(df['starttime_time']>=time_from) & (df['starttime_time']<=time_till)]


In [15]:
#*
df_start_station.groupby(['start station id'])['start station id'].agg(['count']).sort_values(by=['count'], ascending=False).head()

,count
start station id,
519.0,2823
402.0,2368
426.0,2217
459.0,2103
477.0,1994


In [ ]:
#*
df['end_hour'] = df['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)
df['start_hour'] = df['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
                                        time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)
df[(df.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head()

# Выберите идентификаторы станций(end station id), в которые приезжают с 6 до 10 утра включительно

In [16]:
from datetime import datetime, time

time_from = time(6, 00)
time_till = time(10, 00)

df['endtime_time'] = df.apply(lambda x: 
          datetime.strptime(x['starttime'],'%Y-%m-%d %H:%M:%S.%f').time()
          ,axis=1)

df_end_station = df[(df['endtime_time']>=time_from) & (df['endtime_time']<=time_till)]


In [17]:
values = [369, 3106, 3116, 3140]
df_end_station[df_end_station['end station id'].isin(values)]['end station id'].unique()

array([3116., 3140.,  369., 3106.])

In [ ]:
#*
end_stations = [3140, 3106, 3116, 369]
citybike[(citybike.end_hour.isin([6,7,8,9,10])) &
         (citybike['end station id'].isin(end_stations))]['end station id'].unique()